In [23]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss,Dropout2d
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [15]:
## change ##
batch_size = 16
clip = .25

Prepare data

In [16]:
train_gen = SequenceGenSpec(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [24]:
model = Sequential(Dropout(0.5),
                      DenseBlock(129),
                      Dropout(0.7),
                      DenseBlock(258),
                      Dropout(0.7),
                      Linear(194,1),
                      LeakyReLU(0.1,True),
                      LambdaLayer(lambda x:x.squeeze(2)),
                      Dropout(0.5),
                      Linear(516,128),
                      LeakyReLU(0.1,True),
                      Dropout(0.5),
                      Linear(128,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                     ).to('cuda:0')

In [25]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [26]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [27]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:3.145390501389137, val_loss:3.0017387866973877
epoch:1, train_loss:2.8443311458596816, val_loss:2.8703553676605225
epoch:2, train_loss:2.7616498607855577, val_loss:2.919680595397949
epoch:3, train_loss:2.712630044382352, val_loss:2.926020860671997
epoch:4, train_loss:2.71235750329036, val_loss:3.2643165588378906
Training completed in 31.353027820587158s


Submission

In [30]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [31]:
test_gen = SequenceGenSpecTest(submission.seg_id.tolist())
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [32]:
submission.iloc[:,1] = predict(model,test_gen)

In [33]:
submission.to_csv('../Submission/sub_spec_2d_linear.csv',index=False)